# SWB Prospect Theory Modeling


Created: 06/20/2023 by Alie Fink

In [1]:
import numpy as np
import random
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.optimize import minimize # minimize function is used for parameter recovery 
import seaborn as sns 
import tqdm
from scipy.stats import pearsonr
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/scripts/')
import SWB_modeling_utils 

## Find Optim Params

In [4]:
swb_dir = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/'
subj_list = pd.read_excel(f'{swb_dir}SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0])
subj_ids = list(subj_list.PatientID)

In [5]:
subj_ids

['DA8',
 'DA9',
 'DA10',
 'DA11',
 'DA023',
 'MS002',
 'MS003',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030']

### Model 1: Base PT Model
- free params: risk aversion, loss aversion, temp
- bounds = (0.1,3),(0.1,5),(0.1,20)
- beta distrubution assumptions = [1,9],[2,4],[1,8]

distribution assumptions estimated from Charpentier et al 2016 DATA_for_JASP_Anxiety.xlsx


In [90]:
subjects = pd.read_excel('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/SWB_subjects.xlsx', sheet_name=0,usecols='A')
behav_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/behavior_analysis/behavior_preprocessed/'

In [91]:
risk_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=10, lower_bound=0, method='beta',beta_shape=[1,9])
loss_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=10, lower_bound=1, method='beta',beta_shape=[2,4])
temp_inits = SWB_modeling_utils.param_init(n_values=1, n_iter=5, upper_bound=20, lower_bound=0, method='beta',beta_shape=[1,8])

In [112]:
swb_base_pt_dict = {}
bounds=(0.001,3),(0.001,10),(0.01,10)

for subj_id in subj_ids:
    df = pd.read_csv(f'{behav_path}{subj_id}_task_data')

    risk_est, loss_est, temp_est, bic_est, optim_inits = SWB_modeling_utils.run_base_pt(df,risk_inits,loss_inits,temp_inits,bounds=bounds)
    if risk_est == 0:
        print(subj_id)
    swb_base_pt_dict[subj_id] = {'risk_estimation':risk_est,
                                'loss_estimation':loss_est,
                                'temp_estimation':temp_est,
                                'bic_estimation':bic_est,
                                'optim_inits':optim_inits}


In [113]:
swb_base_pt_dict

{'DA8': {'risk_estimation': 1.212120867219247,
  'loss_estimation': 5.1111175595006175,
  'temp_estimation': 2.6255572967724925,
  'bic_estimation': 125.98468083971123,
  'optim_inits': (0.38105460964695825,
   1.9934191488819408,
   2.9612022490488488)},
 'DA9': {'risk_estimation': 1.4843856469190957,
  'loss_estimation': 0.8368549877592054,
  'temp_estimation': 3.2610524801197536,
  'bic_estimation': 192.1165874693769,
  'optim_inits': (2.171815808936534, 3.985094499857248, 1.8483527612040884)},
 'DA10': {'risk_estimation': 0.7817681300920752,
  'loss_estimation': 1.7271895526565089,
  'temp_estimation': 3.058602859807065,
  'bic_estimation': 209.2782799012511,
  'optim_inits': (2.171815808936534, 4.653471118249209, 2.9612022490488488)},
 'DA11': {'risk_estimation': 1.1996249348556003,
  'loss_estimation': 0.22625533466419614,
  'temp_estimation': 1.5277500687064884,
  'bic_estimation': 277.9886762313507,
  'optim_inits': (0.38105460964695825, 3.985094499857248, 1.7884918999238608)},

In [120]:
subj_ids

['DA8',
 'DA9',
 'DA10',
 'DA11',
 'DA023',
 'MS002',
 'MS003',
 'MS016',
 'MS017',
 'MS019',
 'MS022',
 'MS025',
 'MS026',
 'MS027',
 'MS029',
 'MS030']

In [119]:
risk_list = []
loss_list = []
temp_list = []

for subj_id in subj_ids:
    risk_list.append(swb_base_pt_dict[subj_id]['risk_estimation'])
    loss_list.append(swb_base_pt_dict[subj_id]['loss_estimation'])
    temp_list.append(swb_base_pt_dict[subj_id]['temp_estimation'])


In [121]:
pt_params_df = pd.DataFrame(list(zip(subj_ids,risk_list,loss_list,temp_list)),
                            columns=['PatientID','Risk_Aversion','Loss_Aversion','Inverse_Temperature']

)

In [124]:
pt_params_df.to_csv('/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/swb_base_pt_params.csv',index=False)

In [115]:
import pickle 
save_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/'

# create a binary pickle file 
f = open(f'{save_path}/swb_pt_optim_06232023.pkl',"wb")
# # write the python object (dict) to pickle file
pickle.dump(swb_base_pt_dict,f)
# # close file
f.close()

## Run pt models with optim params

In [116]:
swb_base_pt_dfs = {}
bounds=(0.001,3),(0.001,10),(0.001,10)

for subj_id in subj_ids:
    df = pd.read_csv(f'{behav_path}{subj_id}_task_data')
    optim_risk = swb_base_pt_dict[subj_id]['risk_estimation']
    optim_loss = swb_base_pt_dict[subj_id]['loss_estimation']
    optim_temp = swb_base_pt_dict[subj_id]['temp_estimation']
    params = (optim_risk,optim_loss,optim_temp)

    pt_df = SWB_modeling_utils.fit_base_pt(params, df)
    swb_base_pt_dfs[subj_id] = pt_df

In [117]:
pt_df

,tr,choice,choice_prob,choice_pred,choice_pred_prob,util_gamble,util_safe,p_gamble,p_safe
0,0,gamble,0.323013,safe,0.676987,0.372833,0.510953,0.323013,0.676987
1,1,gamble,0.499058,safe,0.500942,-0.000704,0.000000,0.499058,0.500942
2,2,safe,0.669531,safe,0.669531,-0.605324,-0.473530,0.330469,0.669531
3,3,gamble,0.499471,safe,0.500529,-0.000395,0.000000,0.499471,0.500529
4,4,gamble,0.525044,gamble,0.525044,-0.669816,-0.688530,0.525044,0.474956
...,...,...,...,...,...,...,...,...,...
145,145,gamble,0.710376,gamble,0.710376,0.769200,0.601726,0.710376,0.289624
146,146,gamble,0.459939,safe,0.540061,-0.503506,-0.473530,0.459939,0.540061
147,147,gamble,0.607895,gamble,0.607895,-0.705107,-0.786953,0.607895,0.392105
148,148,gamble,0.294886,safe,0.705114,-0.747387,-0.584663,0.294886,0.705114


In [118]:
import pickle 
save_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/'

# create a binary pickle file 
f = open(f'{save_path}/swb_pt_dfs_06232023.pkl',"wb")
# # write the python object (dict) to pickle file
pickle.dump(swb_base_pt_dfs,f)
# # close file
f.close()

## Update model data for GLM inputs

In [123]:
model_input_path = '/Users/alexandrafink/Documents/GraduateSchool/SaezLab/SWB/swb_computational_modeling/swb_behav_models/data/model_input_data_06192023'
model_input = pd.read_csv(model_input_path)

In [ ]:
def get_model_data(subj_id,task_df,rate_df):
    model_data_dict = {}

    #get rating info
    round = rate_df['Round'][max(loc for loc, val in enumerate(rate_df['Round']) if val == 1)+1:] #need index of last round 1 because some pts have multiple round 1 scores, start after last round 1 index
    rate = rate_df['Rating'][max(loc for loc, val in enumerate(rate_df['Round']) if val == 1)+1:]
    zscore_rate = rate_df['zscore_mood'][max(loc for loc, val in enumerate(rate_df['Round']) if val == 1)+1:]


    cr1 = []
    cr2 = []
    cr3 = []
    ev1 = []
    ev2 = []
    ev3 = []
    rpe1 = []
    rpe2 = []
    rpe3 = []
    tcpe1 = []
    tcpe2 = []
    tcpe3 = []
    dcpe1 = []
    dcpe2 = []
    dcpe3 = []
    treg1 = []
    treg2 = []
    treg3 = []
    dreg1 = []
    dreg2 = []
    dreg3 = []
    trel1 = []
    trel2 = []
    trel3 = []
    drel1 = []
    drel2 = []
    drel3 = []

    for r in round:
        #index for task df
        t3 = r-4 #t-3 trial 
        t2 = r-3 #t-2 trial
        t1 = r-2 #t-1 trial
        
        cr1.append(task_df['CR'][t1])
        cr2.append(task_df['CR'][t1])
        cr3.append(task_df['CR'][t3])
        ev1.append(task_df['choiceEV'][t1])
        ev2.append(task_df['choiceEV'][t2])
        ev3.append(task_df['choiceEV'][t3])
        rpe1.append(task_df['RPE'][t1])
        rpe2.append(task_df['RPE'][t2])
        rpe3.append(task_df['RPE'][t3])
        tcpe1.append(task_df['totalCPE'][t1])
        tcpe2.append(task_df['totalCPE'][t2])
        tcpe3.append(task_df['totalCPE'][t3])
        dcpe1.append(task_df['decisionCPE'][t1])
        dcpe2.append(task_df['decisionCPE'][t2])
        dcpe3.append(task_df['decisionCPE'][t3])
        treg1.append(task_df['totalRegret'][t1])
        treg2.append(task_df['totalRegret'][t2])
        treg3.append(task_df['totalRegret'][t3])
        dreg1.append(task_df['decisionRegret'][t1])
        dreg2.append(task_df['decisionRegret'][t2])
        dreg3.append(task_df['decisionRegret'][t3])
        trel1.append(task_df['totalRelief'][t1])
        trel2.append(task_df['totalRelief'][t2])
        trel3.append(task_df['totalRelief'][t3])
        drel1.append(task_df['decisionRelief'][t1])
        drel2.append(task_df['decisionRelief'][t2])
        drel3.append(task_df['decisionRelief'][t3])

    
    
    
    model_data_dict['subj_id'] = [subj_id]*50
    model_data_dict['round'] = round
    model_data_dict['rate'] = rate
    model_data_dict['zscore_rate'] = zscore_rate
    model_data_dict['cr(t-1)'] = cr1
    model_data_dict['cr(t-2)'] = cr2
    model_data_dict['cr(t-3)'] = cr3
    model_data_dict['choice_ev(t-1)'] = ev1
    model_data_dict['choice_ev(t-2)'] = ev2
    model_data_dict['choice_ev(t-3)'] = ev3
    model_data_dict['rpe(t-1)'] = rpe1
    model_data_dict['rpe(t-2)'] = rpe2
    model_data_dict['rpe(t-3)'] = rpe3
    model_data_dict['totalcpe(t-1)'] = tcpe1
    model_data_dict['totalcpe(t-2)'] = tcpe2
    model_data_dict['totalcpe(t-3)'] = tcpe3
    model_data_dict['decisioncpe(t-1)'] = dcpe1
    model_data_dict['decisioncpe(t-2)'] = dcpe2
    model_data_dict['decisioncpe(t-3)'] = dcpe3
    model_data_dict['totalregret(t-1)'] = treg1
    model_data_dict['totalregret(t-2)'] = treg2
    model_data_dict['totalregret(t-3)'] = treg3
    model_data_dict['decisionregret(t-1)'] = dreg1
    model_data_dict['decisionregret(t-2)'] = dreg2
    model_data_dict['decisionregret(t-3)'] = dreg3
    model_data_dict['totalrelief(t-1)'] = trel1
    model_data_dict['totalrelief(t-2)'] = trel2
    model_data_dict['totalrelief(t-3)'] = trel3
    model_data_dict['decisionrelief(t-1)'] = drel1
    model_data_dict['decisionrelief(t-2)'] = drel2
    model_data_dict['decisionrelief(t-3)'] = drel3

    
    
    return model_data_dict